In [1]:
import sys
import os
import pandas as pd
import re

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [12]:
nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [13]:
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")


In [14]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [15]:
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]

In [16]:
exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']

In [17]:
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [18]:
test_train_df

,sub_label,SplitGroup,AssignedInJune,SubjectId,Task,Include
1,DEV004,Holdout,True,NaN,NaN,True
2,DEV005,Train,True,NaN,NaN,True
3,DEV006,Train,True,NaN,NaN,True
4,DEV008,Holdout,True,NaN,NaN,True
5,DEV009,Train,True,NaN,NaN,True
...,...,...,...,...,...,...
184,DEV221,Train,False,NaN,NaN,True
185,DEV222,Holdout,False,NaN,NaN,True
186,DEV223,Holdout,False,NaN,NaN,True
187,DEV224,Holdout,False,NaN,NaN,True


In [19]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [20]:
from dev_wtp_io_utils import import_sst_cond_w1_subjs_to_pkl

In [11]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:2],'conditions',
    out_folder = ml_data_folderpath + "/")

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   
2                2      FailedStop   
3                3             Cue   
4                4        FailedGo   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV005     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV005     1  
2   spm_spm:beta (0003) - Sn(1) FailedStop*bf(1)  beta_0003.nii  DEV005     1  
3          spm_spm:beta (0004) - Sn(1) Cue*bf(1)  beta_0004.nii  DEV005     1  
4     spm_spm:beta (0005) - Sn(1) FailedGo*bf(1)  beta_0005.nii  DEV005     1  
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting

OK, not bad, so we have learned to load data, and show it. We might be best to re-write the code, though, to use the first image as the image series's own mask--using `nilearn.masking.compute_brain_mask`, rather than worry about why the input mask isn't quite aligned. 

In [ ]:
len(train_subjs)

### With supplementary data

In [21]:
individual_differences = pd.read_csv(ml_data_folderpath + "/data_by_ppt.csv")
individual_differences = individual_differences.rename(columns={'SID':'subject'})
individual_differences['wave']=1
#individual_differences['wave'] = individual_differences['wave'].astype(object) # for compatibility with the wave column in the dataset

In [22]:
from outlier_detection import *

In [23]:
"', '".join(individual_differences.columns)

"subject', 'cancer_promoting_minus_preventing_FCI', 'cancer_promoting_minus_preventing_FFQ', 'cancer_promoting_minus_preventing_craved_FCI', 'cancer_promoting_minus_preventing_craved_FFQ', 'cancer_promoting_minus_preventing_liked_FCI', 'cancer_promoting_minus_preventing_liked_FFQ', 'cancer_promoting_FCI', 'cancer_promoting_FFQ', 'cancer_preventing_FCI', 'cancer_preventing_FFQ', 'BSCS', 'cSES', 'EDM', 'BIS_11', 'PCS', 'RS', 'TRSQ', 'bf_1', 'weight_0', 'height_0', 'birthsex', 'age365', 'bmi_0', 'bmi_1', 'bmi', 'bf_1_controlled', 'bf_1_bsexnormedzs', 'ACES_sum', 'ACES_household_dysfunction', 'ACES_neglectful_parenting', 'ACES_abuse', 'ACES_divorced_separated', 'BFI_agreeableness', 'BFI_conscientiousness', 'BFI_extraversion', 'BFI_neuroticism', 'BFI_openness', 'DEMO_mcarthur_social_standing', 'IMI_effort_importance', 'IMI_interest_enjoyment', 'IMI_perceived_choice', 'IMI_perceived_competence', 'IMI_value_usefulness', 'IPAQ_moderateminutes', 'IPAQ_sittinghours', 'IPAQ_vigorousminutes', 'IPA

In [24]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq


train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [25]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_spatially_concatenated',
    concatenate_condition_labels=True
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV005     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV005     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV005.nii
  subject  wave
0  DEV005     1
...imported data.
temp mask deleted
DEV006
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

               

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV019.nii
  subject  wave
0  DEV019     1
...imported data.
temp mask deleted
DEV020
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV020     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV020     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/g

KeyboardInterrupt: 

In [32]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_w_supplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV005     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV005     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV005     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.ni

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV017     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV017     1  
...imported data.
temp mask deleted
DEV018
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV018     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV018     1  
.....verified that all expect

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV027     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV027     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV027     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV027     1  
...imported data.
temp mask deleted
DEV028
   condition_index condition_

temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV040     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV040     1  
...imported data.
temp mask deleted
DEV041
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV041     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV041     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading fi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV051     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV051     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV051     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV051     1  
...imported data.
temp mask deleted
DEV052
   condition_index condition_

temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV060     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV060     1  
...imported data.
temp mask deleted
DEV062
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV062     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV062     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading fi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV076     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV076     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV076     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV076     1  
...imported data.
temp mask deleted
DEV077
   condition_index condition_

temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV089     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV089     1  
...imported data.
temp mask deleted
DEV097
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV097     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV097     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading fi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV159     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV159     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV159     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV159     1  
...imported data.
temp mask deleted
DEV164
   condition_index condition_

temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV206     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV206     1  
...imported data.
temp mask deleted
DEV215
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV215     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV215     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading fi

## concatenating conditions

In [ ]:
from nilearn import plotting
from nilearn.masking import compute_brain_mask
import nltools as nlt
import nilearn as nil

In [ ]:
from nltools import Brain_Data

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:2],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_spatially_concatenated',
    concatenate_condition_labels=True
)

In [ ]:
out_data = pickle.load(open(
    '/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_2subs_correct_cond_w_supplementary_mask_fix.pkl','rb'
))

In [ ]:
out_data.shape

In [ ]:
plotting.plot_glass_brain(out_data[0,].to_nifti())

## post-error conditions

In [27]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq

train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [28]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:2],'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_w_supplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV005   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV005   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:2],'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_go_fcs_w_supplementary'
)

In [29]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_correct_stop_cg_w_supplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV005   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV005   

   wave  
0     1  
...imported data.
temp mask deleted
DEV006
   condition_index                condition_label  \
0              

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV018   

   wave  
0     1  
...imported data.
temp mask deleted
DEV019
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV019   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description         

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV029   

   wave  
0     1  
...imported data.
temp mask deleted
DEV030
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV030   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV046   

   wave  
0     1  
...imported data.
temp mask deleted
DEV047
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV047   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV057   

   wave  
0     1  
...imported data.
temp mask deleted
DEV058
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV058   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV073   

   wave  
0     1  
...imported data.
temp mask deleted
DEV074
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV074   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV089   

   wave  
0     1  
...imported data.
temp mask deleted
DEV097
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV097   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV164   

   wave  
0     1  
...imported data.
temp mask deleted
DEV169
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV169   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV217   

   wave  
0     1  
...imported data.
temp mask deleted
DEV218
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV218   

   wave  
0     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may

In [30]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingFailedStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_failed_stop_cg_failed_stop_w_supplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV005   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV005   

   wave  
1     1  
...imported data.
temp mask deleted
DEV006
   condition_index               condition_label  \
1                1  

   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV017   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV017   

   wave  
1     1  
...imported data.
temp mask deleted
DEV018
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta

   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV027   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV027   

   wave  
1     1  
...imported data.
temp mask deleted
DEV028
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta

   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV042   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV042   

   wave  
1     1  
...imported data.
temp mask deleted
DEV043
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV055   

   wave  
1     1  
...imported data.
temp mask deleted
DEV056
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV056   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may tak

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV069   

   wave  
1     1  
...imported data.
temp mask deleted
DEV071
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV071   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may tak

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV086   

   wave  
1     1  
...imported data.
temp mask deleted
DEV087
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV087   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may tak

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV158   

   wave  
1     1  
...imported data.
temp mask deleted
DEV159
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV159   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may tak

....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV215   

   wave  
1     1  
...imported data.
temp mask deleted
DEV216
   condition_index               condition_label  \
1                1  CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV216   

   wave  
1     1  
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
temp mask created.
loading files. This step may tak

In [31]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_stop_cg_spatially_concatenated',
    concatenate_condition_labels=True
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV005   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV005   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV005.nii
  subject  wave
0  DEV005     1
...imported data.
temp mask deleted
DEV006
   condition_index                con

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV018.nii
  subject  wave
0  DEV018     1
...imported data.
temp mask deleted
DEV019
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV019   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV019   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

  subject  wave
0  DEV028     1
...imported data.
temp mask deleted
DEV029
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV029   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV029   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV029.nii
  subject  wave
0  DEV029     1
...imported data.
temp mask deleted
DEV030
   condition_index                condition_label  \
0        

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV043.nii
  subject  wave
0  DEV043     1
...imported data.
temp mask deleted
DEV046
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV046   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV046   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV056.nii
  subject  wave
0  DEV056     1
...imported data.
temp mask deleted
DEV057
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV057   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV057   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV071.nii
  subject  wave
0  DEV071     1
...imported data.
temp mask deleted
DEV073
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV073   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV073   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV087   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV087   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV087.nii
  subject  wave
0  DEV087     1
...imported data.
temp mask deleted
DEV089
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoF

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV158.nii
  subject  wave
0  DEV158     1
...imported data.
temp mask deleted
DEV159
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV159   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV159   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat

.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask created.
loading files. This step may take some time...
converting to nifti and fdata
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/temp_mask_get_Brain_Data_betas_for_sub_DEV215.nii
  subject  wave
0  DEV215     1
...imported data.
temp mask deleted
DEV216
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV216   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV216   

   wave  
0     1  
1     1  
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
temp mask creat